# Day 20 - Trench Map

## Read input


In [93]:
from utils import read_multisection_input


def transform_image(section):
    lines = section.split('\n')
    image = {}
    for y in range(len(lines)): 
        for x in range(len(lines[y])):
            image[(x,y)] = lines[y][x]
            
    return image

VALUES = {
    '#': '1',
    '.': '0'
}

algorithm, image = read_multisection_input(20, [str, transform_image], example=True)

In [87]:
 def get_points(point):
    x,y = point
    return [
        (x-1, y-1),
        (x, y-1),
        (x+1, y-1),
        (x-1, y),
        (x, y),
        (x+1, y),
        (x-1, y+1),
        (x, y+1),
        (x+1, y+1)
    ]

In [94]:
def get_new_value(point, image, algorithm):
    points = get_points(point)
    binary = ''
    for _point in points:
        p = image.get(_point, '.')
        binary += VALUES[p]
    
    new_point = algorithm[int(binary, 2)]
    
    return new_point

In [89]:
def print_image(image):
    max_x, max_y = max(x for x, y in image.keys()), max(y for x, y in image.keys())
    for y in range(max_y+1):
        for x in range(max_x+1):
            print(image[(x,y)], end="")
            
        print()

In [90]:
def enhance(image, algorithm):
    new_image = {}
    max_x, max_y = max(x for x, y in image.keys()), max(y for x, y in image.keys())
    for y in range(max_y+1):
        for x in range(max_x+1):
             new_image[(x,y)] = get_new_value((x,y), image, algorithm)
                
    return new_image
    

In [96]:
print_image(enhance(image, algorithm))

...............
...............
...............
...............
.....##.##.....
....#..#.#.....
....#..#.##....
....####..#....
......#.#......
......##..#....
........###....
...............
...............
...............
...............
